In [1]:
# initialize findspark for local
import findspark
findspark.init('/Users/johnrick/opt/spark-2.4.7-bin-hadoop2.7')

In [2]:
import configparser
from datetime import datetime
# import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

In [3]:
# Configure the necessary Spark environment
import os
import sys
import boto3

In [4]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID'] = config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = config['AWS']['AWS_SECRET_ACCESS_KEY']
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk-pom:1.10.34,org.apache.hadoop:hadoop-aws:2.7.1 pyspark-shell'

In [5]:
pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", "")
if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")

# Add the py4j to the path.
# You may need to change the version number to match your install
sys.path.insert(0, os.path.join(spark_home, "/Users/johnrick/opt/spark-2.4.7-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip"))

# Initialize PySpark
exec(open(os.path.join(spark_home, "/Users/johnrick/Downloads/spark-3.0.0-preview2-bin-hadoop2.7/python/pyspark/python/pyspark/shell.py")).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.7.4 (default, Aug 13 2019 15:17:50)
SparkSession available as 'spark'.


In [6]:
from pyspark.sql.types import StructType as R, StructField, DoubleType, StringType, IntegerType, DateType, TimestampType, StructType, LongType

In [7]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

In [8]:
song_schema = StructType([
        StructField("artist_id", StringType(), False),
        StructField("artist_latitude", StringType(), True),
        StructField("artist_longitude", StringType(), True),
        StructField("artist_location", StringType(), True),
        StructField("artist_name", StringType(), False),
        StructField("song_id", StringType(), False),
        StructField("title", StringType(), False),
        StructField("duration", DoubleType(), False),
        StructField("year", IntegerType(), False)
])

In [9]:
ls

README.md         dl.cfg            get_data.ipynb    ud-s3datalake/
data/             etl.py            test_local.ipynb  ud-s3datalake-ap/


In [14]:
song_data = "data/song_data/*/*/*/*.json"

In [15]:
df = spark.read.json(song_data, schema=song_schema)

In [20]:
# extract columns to create songs table
songs_table = df.select("song_id",
                       "title", 
                       "artist_id",
                       "year",
                       "duration").dropDuplicates()

In [21]:
# extract column to get artists_table
artists_table = df.select("artist_id",
                         col("artist_name").alias("name"),
                         col("artist_location").alias("location"),
                         col("artist_latitude").alias("latitude"),
                         col("artist_longitude").alias("longitude")).dropDuplicates()

# Process Log Data

In [22]:
log_schema = StructType([
        StructField("artist", StringType(), True),
        StructField("auth", StringType(), False),
        StructField("firstName", StringType(), True),
        StructField("gender", StringType(), True),
        StructField("itemInSession", IntegerType(), False),
        StructField("lastName", StringType(), True),
        StructField("length", DoubleType(), True),
        StructField("level", StringType(), False),
        StructField("location", StringType(), True),
        StructField("method", StringType(), False),
        StructField("page", StringType(), False),
        StructField("registration", DoubleType(), True),
        StructField("sessionId", IntegerType(), False),
        StructField("song", StringType(), True),
        StructField("status", IntegerType(), False),
        StructField("ts", DoubleType(), False),
        StructField("userAgent", StringType(), True),
        StructField("userId", StringType(), True)
    ])

In [23]:
log_data = "data/log-data/*.json"

In [24]:
# read log data file
log_df = spark.read.json(log_data, schema = log_schema)

In [25]:
log_df = log_df.where(log_df.page == "NextSong")

In [26]:
users_table = log_df.select(col("userId").alias("user_id"),
                       col("firstName").alias("first_name"),
                       col("lastName").alias("last_name"),
                       "gender",
                       "level").dropDuplicates()

In [27]:
users_table.limit(5).toPandas()

,user_id,first_name,last_name,gender,level
0,26,Ryan,Smith,M,free
1,7,Adelyn,Jordan,F,free
2,71,Ayleen,Wise,F,free
3,81,Sienna,Colon,F,free
4,87,Dustin,Lee,M,free


## time_stamp

In [28]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))

In [30]:
log_df = log_df.withColumn("timestamp", get_timestamp(log_df.ts))

In [31]:
log_df.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,timestamp
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1.542242e+12,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-14 19:30:26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1.542242e+12,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-14 19:41:21
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1.542243e+12,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-14 19:45:41
3,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,597,Blackbird,200,1.542253e+12,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61,2018-11-14 22:44:09
4,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,602,Best Of Both Worlds (Remastered Album Version),200,1.542261e+12,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 00:48:55


In [32]:
# create datetime column from original timestamp column
get_datetime = udf(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d'))
log_df = log_df.withColumn("datetime", get_datetime(log_df.ts))

In [33]:
log_df.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,timestamp,datetime
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1.542242e+12,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-14 19:30:26,2018-11-14
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1.542242e+12,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-14 19:41:21,2018-11-14
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1.542243e+12,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-14 19:45:41,2018-11-14
3,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,597,Blackbird,200,1.542253e+12,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61,2018-11-14 22:44:09,2018-11-14
4,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,602,Best Of Both Worlds (Remastered Album Version),200,1.542261e+12,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 00:48:55,2018-11-15


In [34]:
# extract columns to create time table
time_table = log_df.select(
    log_df.timestamp.alias('start_time'),
    hour(log_df.datetime).alias('hour'),
    dayofmonth(log_df.datetime).alias('day'),
    weekofyear(log_df.datetime).alias('week'),
    month(log_df.datetime).alias('month'),
    year(log_df.datetime).alias('year'),
    date_format(log_df.datetime, 'u').alias('weekday')).dropDuplicates().limit(10)

In [35]:
time_table.limit(5).toPandas()

,start_time,hour,day,week,month,year,weekday
0,2018-11-15 01:14:16,0,15,46,11,2018,4
1,2018-11-15 05:57:10,0,15,46,11,2018,4
2,2018-11-15 09:22:29,0,15,46,11,2018,4
3,2018-11-15 09:46:26,0,15,46,11,2018,4
4,2018-11-15 11:18:45,0,15,46,11,2018,4


In [ ]:
# songs_table,
# artists_table,
# users_table,
# time_table

In [37]:
songs_logs = log_df.join(songs_table, (log_df.song == songs_table.title))

In [38]:
artists_songs_logs = songs_logs.join(artists_table, (songs_logs.artist == artists_table.name))